In [1]:
import numpy as np

#### data titanic tanpa fillna dan get_dummies

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("C:\\Users\\Gustav\\Desktop\\Data Projects\\supervised_learning-master\\04 - KNN & Scikit-learn\\data\\titanic.csv", index_col="PassengerId")
df.drop(columns=["Name","Ticket","Age","Cabin"],inplace=True)

In [4]:
X = df.drop(columns="Survived")
y = df.Survived


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 6), (179, 6), (712,), (179,))

### Preprocessor

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder

In [6]:
numerical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", MinMaxScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder())
])

In [7]:
X_train.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,
693,3,male,0,0,56.4958,S
482,2,male,0,0,0.0000,S
528,1,male,0,0,221.7792,S
856,3,female,0,1,9.3500,S
802,2,female,1,1,26.2500,S


In [8]:
from sklearn.compose import ColumnTransformer

In [9]:
preprocessor = ColumnTransformer([
    ("numeric", numerical_pipeline, ["SibSp","Parch","Fare"]),
    ("categoric", categorical_pipeline, ["Pclass","Sex","Embarked"])
])

### Pipeline

In [10]:
from sklearn.neighbors import KNeighborsClassifier

In [11]:
pipeline = Pipeline([
    ("prep", preprocessor),
    ("algo", KNeighborsClassifier())
])

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['SibSp', 'Parch', 'Fare']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Pclass', 'Sex',
                                                   'Embarked'])])),
                ('algo', KNeighborsClassifier())])

In [13]:
pipeline.score(X_test, y_test)

0.776536312849162

### Grid Search Cross Validation

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
parameter = {
    "algo__n_neighbors": range(1,51,2),
    "algo__weights": ["uniform", "distance"],
    "algo__p" : [1,2]
}

model = GridSearchCV(pipeline, parameter, cv=3, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['SibSp',
                                                                          'Parch',
                                                                          'Fare']),
                                                                        ('categoric',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder())]),
                                                                         ['Pclass',
                                                                          'Sex',
                                                                          'Embarked'])])),
                                       ('algo', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'algo__n_neighbors': range(1, 51, 2),
                         'algo__p': [1, 2],
                         'algo__weights': ['uniform', 'distance']},
             verbose=1)

In [16]:
pd.DataFrame(model.cv_results_).sort_values("rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algo__n_neighbors,param_algo__p,param_algo__weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
40,0.051637,0.008801,0.043643,0.006547,21,1,uniform,"{'algo__n_neighbors': 21, 'algo__p': 1, 'algo_...",0.815126,0.818565,0.810127,0.814606,0.003465,1
36,0.049971,0.021264,0.111602,0.009871,19,1,uniform,"{'algo__n_neighbors': 19, 'algo__p': 1, 'algo_...",0.815126,0.818565,0.805907,0.813200,0.005344,2
24,0.053969,0.013359,0.042309,0.002054,13,1,uniform,"{'algo__n_neighbors': 13, 'algo__p': 1, 'algo_...",0.819328,0.810127,0.810127,0.813194,0.004337,3
28,0.038312,0.006795,0.064962,0.012825,15,1,uniform,"{'algo__n_neighbors': 15, 'algo__p': 1, 'algo_...",0.819328,0.810127,0.810127,0.813194,0.004337,3
38,0.022655,0.001698,0.085284,0.008253,19,2,uniform,"{'algo__n_neighbors': 19, 'algo__p': 2, 'algo_...",0.815126,0.814346,0.805907,0.811793,0.004174,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,0.043641,0.010333,0.027650,0.008952,3,1,distance,"{'algo__n_neighbors': 3, 'algo__p': 1, 'algo__...",0.760504,0.734177,0.751055,0.748579,0.010890,95
3,0.040644,0.005555,0.039976,0.017099,1,2,distance,"{'algo__n_neighbors': 1, 'algo__p': 2, 'algo__...",0.752101,0.683544,0.772152,0.735932,0.037938,97
2,0.043307,0.017510,0.050637,0.006233,1,2,uniform,"{'algo__n_neighbors': 1, 'algo__p': 2, 'algo__...",0.752101,0.683544,0.772152,0.735932,0.037938,97
1,0.060630,0.016348,0.023320,0.007538,1,1,distance,"{'algo__n_neighbors': 1, 'algo__p': 1, 'algo__...",0.747899,0.683544,0.763713,0.731719,0.034671,99


In [17]:
model.best_params_

{'algo__n_neighbors': 21, 'algo__p': 1, 'algo__weights': 'uniform'}

In [18]:
model.score(X_train, y_train), model.score(X_test, y_test)

(0.8174157303370787, 0.7821229050279329)

### Prediksi

In [21]:
data = [
    [1,"female",1,1,80,"S"],
    [3,"male",0,0,5,"S"]
]
X_pred = pd.DataFrame(data, index=["Rose","Jack"], columns=X.columns)
X_pred

,Pclass,Sex,SibSp,Parch,Fare,Embarked
Rose,1,female,1,1,80,S
Jack,3,male,0,0,5,S


In [22]:
model.predict(X_pred)

array([1, 0], dtype=int64)

In [23]:
X_pred["Survived"] = model.predict(X_pred)
X_pred

,Pclass,Sex,SibSp,Parch,Fare,Embarked,Survived
Rose,1,female,1,1,80,S,1
Jack,3,male,0,0,5,S,0


### Pickling object in python

In [29]:
import pickle,os

In [30]:
os.makedirs("folder_model", exist_ok=True)

In [25]:
pickle.dump(model,open("knn.pkl","wb")) # write binary


In [27]:
model = pickle.load(open("knn.pkl", "rb")) # read binary